## **2D FDTD TM<sup>Z</sup> TFSF Solver**

Python adaptation of John B. Schneider's C program from section 8.6 of his textbook *Understanding the Finite-Difference Time-Domain
Method.*  This program simulates a TM<sup>Z</sup> 2D FDTD grid with a TFSF source located at the left side of the grid.  Second-order ABC boundary conditions are placed around the perimeter of the grid.

Chapter 8 of Schneider's book is located [HERE](https://eecs.wsu.edu/~schneidj/ufdtd/chap8.pdf) and his GitHub source code is available [HERE](https://github.com/john-b-schneider/uFDTD).

In [ ]:
import numpy as np


class oneDGrid: pass
class teZGrid: pass
class tmZGrid: pass
class threeDGrid: pass


class Grid:
    def __init__(self):
        self.Hx, self.Chxh, self.Chxe = None, None, None
        self.Hy, self.Chyh, self.Chye = None, None, None
        self.Hz, self.Chzh, self.Chze = None, None, None
        self.Ex, self.Cexe, self.Cexh = None, None, None
        self.Ey, self.Ceye, self.Ceyh = None, None, None
        self.Ez, self.Ceze, self.Cezh = None, None, None
        self.SizeX, self.SizeY, self.SizeZ = None, None, None
        self.Time, self.MaxTime = None, None
        self.Type = None
        self.Cdtds = None


def gridInit(g):
    imp0 = 377.0
    g.Type = tmZGrid
    g.SizeX = SIZEX               # x size of domain
    g.SizeY = SIZEY               # y size of domain
    g.MaxTime = MAXTIME           # duration of simulation
    g.Cdtds = 1.0 / np.sqrt(2.0)  # Courant number
    g.Time = 0
    
    g.Hx = np.zeros((g.SizeX, g.SizeY-1))
    g.Chxh = np.ones((g.SizeX, g.SizeY-1))
    g.Chxe = np.ones((g.SizeX, g.SizeY-1)) * g.Cdtds / imp0
    g.Hy = np.zeros((g.SizeX-1, g.SizeY))
    g.Chyh = np.ones((g.SizeX-1, g.SizeY))
    g.Chye = np.ones((g.SizeX-1, g.SizeY)) * g.Cdtds / imp0
    g.Ez = np.zeros((g.SizeX, g.SizeY))
    g.Ceze = np.ones((g.SizeX, g.SizeY))
    g.Cezh = np.ones((g.SizeX, g.SizeY)) * g.Cdtds * imp0

In [ ]:
"""Define macros for arrays that store the previous values of the
 * fields.  For each one of these arrays the three arguments are as
 * follows:
 *
 *   first argument:  spatial displacement from the boundary
 *   second argument: displacement back in time
 *   third argument:  distance from either the bottom (if EzLeft or
 *                    EzRight) or left (if EzTop or EzBottom) side
 *                    of grid
 *                    
"""

initDone = 0
coef0, coef1, coef2 = 0, 0, 0
EzLeft, EzRight, EzTop, EzBottom = 0, 0, 0, 0


def abcInit(g):
    """Second-order ABC for TMz grid."""
    global initDone, EzLeft, EzRight, EzTop, EzBottom
    global coef0, coef1, coef2
    initDone = 1

    # allocate memory for ABC arrays
    EzLeft = np.zeros((3, 2, g.SizeY))
    EzRight = np.zeros((3, 2, g.SizeY))
    EzTop = np.zeros((3, 2, g.SizeX))
    EzBottom = np.zeros((3, 2, g.SizeX))

    # calculate ABC coefficients
    temp1 = np.sqrt(g.Cezh[0, 0] * g.Chye[0, 0])
    temp2 = 1.0 / temp1 + 2.0 + temp1
    coef0 = -(1.0 / temp1 - 2.0 + temp1) / temp2
    coef1 = -2.0 * (temp1 - 1.0 / temp1) / temp2
    coef2 = 4.0 * (temp1 + 1.0 / temp1) / temp2


def abc(g):
    # ABC at left side of grid
    for nn in range(g.SizeY):
        g.Ez[0, nn] = coef0 * (g.Ez[2, nn] + EzLeft[0, 1, nn]) \
            + coef1 * (EzLeft[0, 0, nn] + EzLeft[2, 0, nn]
                       - g.Ez[1, nn] - EzLeft[1, 1, nn]) \
            + coef2 * EzLeft[1, 0, nn] - EzLeft[2, 1, nn]

        # memorize old fields
        for mm in range(3):
            EzLeft[mm, 1, nn] = EzLeft[mm, 0, nn]
            EzLeft[mm, 0, nn] = g.Ez[mm, nn]

    # ABC at right side of grid
    for nn in range(g.SizeY):
        g.Ez[g.SizeX - 1, nn] = coef0 * (g.Ez[g.SizeX - 3, nn] + EzRight[0, 1, nn]) \
            + coef1 * (EzRight[0, 0, nn] + EzRight[2, 0, nn]
                       - g.Ez[g.SizeX - 2, nn] - EzRight[1, 1, nn]) \
            + coef2 * EzRight[1, 0, nn] - EzRight[2, 1, nn]

        # memorize old fields
        for mm in range(3):
            EzRight[mm, 1, nn] = EzRight[mm, 0, nn]
            EzRight[mm, 0, nn] = g.Ez[g.SizeX - 1 - mm, nn]

    # ABC at bottom of grid
    for mm in range(g.SizeX):
        g.Ez[mm, 0] = coef0 * (g.Ez[mm, 2] + EzBottom[0, 1, mm]) \
            + coef1 * (EzBottom[0, 0, mm] + EzBottom[2, 0, mm]
                       - g.Ez[mm, 1] - EzBottom[1, 1, mm]) \
            + coef2 * EzBottom[1, 0, mm] - EzBottom[2, 1, mm]

        # memorize old fields
        for nn in range(3):
            EzBottom[nn, 1, mm] = EzBottom[nn, 0, mm]
            EzBottom[nn, 0, mm] = g.Ez[mm, nn]

    # ABC at top of grid
    for mm in range(g.SizeX):
        g.Ez[mm, g.SizeY - 1] = coef0 * (g.Ez[mm, g.SizeY - 3] + EzTop[0, 1, mm]) \
            + coef1 * (EzTop[0, 0, mm] + EzTop[2, 0, mm]
                       - g.Ez[mm, g.SizeY - 2] - EzTop[1, 1, mm]) \
            + coef2 * EzTop[1, 0, mm] - EzTop[2, 1, mm]

        # memorize old fields
        for nn in range(3):
            EzTop[nn, 1, mm] = EzTop[nn, 0, mm]
            EzTop[nn, 0, mm] = g.Ez[mm, g.SizeY - 1 - nn]

In [ ]:
NLOSS = 20       # number of lossy cells at end of 1D grid
MAX_LOSS = 0.35  # maximum loss factor in lossy layer /*@\label{grid1dezX}@*/

def gridInit1d(g):
    imp0 = 377.0
  
    g.SizeX += NLOSS    # size of domain
    g.Type = oneDGrid   # set grid type

    g.Hy = np.zeros(g.SizeX-1)
    g.Chyh = np.ones(g.SizeX-1)
    g.Chye = np.ones(g.SizeX-1) * g.Cdtds / imp0
    g.Ez = np.zeros(g.SizeX)
    g.Ceze = np.ones(g.SizeX)
    g.Cezh = np.ones(g.SizeX) * g.Cdtds * imp0
      
    # set the electric- and magnetic-field update coefficients
    for mm in range(g.SizeX - 1 - NLOSS, g.SizeX - 1):
        depthInLayer = mm - (g.SizeX - 1 - NLOSS) + 0.5;
        lossFactor = MAX_LOSS * (depthInLayer / NLOSS)**2
        g.Ceze[mm] = (1.0 - lossFactor) / (1.0 + lossFactor)
        g.Cezh[mm] = g.Cdtds * imp0 / (1.0 + lossFactor)
        depthInLayer += 0.5
        lossFactor = MAX_LOSS * (depthInLayer / NLOSS)**2
        g.Chyh[mm] = (1.0 - lossFactor) / (1.0 + lossFactor)
        g.Chye[mm] = g.Cdtds / imp0 / (1.0 + lossFactor);

In [ ]:
from copy import deepcopy
import sys

firstX, firstY = 5, 5  # indices for first point in TF region
lastX, lastY = 95, 75  # indices for last point in TF region

g1 = None  # 1D auxilliary grid

def tfsfInit(g):
    global g1
    g1 = deepcopy(g)  # copy information from 2D array
    gridInit1d(g1)    # initialize 1d grid

    ezIncInit(g)  # initialize source function

def tfsfUpdate(g):
    global g1

    # check if tfsfInit() has been called
    if g1 is None:
        print("tfsfUpdate: tfsfInit must be called before tfsfUpdate.\n"
              "            Boundary location must be set to positive value.\n")
        sys.exit(-1)

    # correct Hy along left edge
    mm = firstX - 1
    for nn in range(firstY, lastY+1):
        g.Hy[mm, nn] -= g.Chye[mm, nn] * g1.Ez[mm + 1]

    # correct Hy along right edge
    mm = lastX
    for nn in range(firstY, lastY+1):
        g.Hy[mm, nn] += g.Chye[mm, nn] * g1.Ez[mm]

    # correct Hx along the bottom
    nn = firstY - 1
    for mm in range(firstX, lastX+1):
        g.Hx[mm, nn] += g.Chxe[mm, nn] * g1.Ez[mm]

    # correct Hx along the top
    nn = lastY
    for mm in range(firstX, lastX+1):
        g.Hx[mm, nn] -= g.Chxe[mm, nn] * g1.Ez[mm]

    updateH2d(g1);    # update 1D magnetic field
    updateE2d(g1);    # update 1D electric field
    g1.Ez[0] = ezInc(g1.Time, 0.0) # set source node
    g1.Time += 1      # increment time in 1D grid

    # correct Ez adjacent to TFSF boundary
    # correct Ez field along left edge
    mm = firstX
    for nn in range(firstY, lastY+1):
        g.Ez[mm, nn] -= g.Cezh[mm, nn] * g1.Hy[mm - 1]

    # correct Ez field along right edge
    mm = lastX
    for nn in range(firstY, lastY+1):
        g.Ez[mm, nn] += g.Cezh[mm, nn] * g1.Hy[mm]

    # no need to correct Ez along top and bottom since
    # incident Hx is zero

In [ ]:
import sys
cdtds = 0
ppw = 0


def ezIncInit(g):
    """Initialize source-function variables."""
    global cdtds, ppw
    cdtds = g.Cdtds
    ppw = 20


def ezInc(time, location):
    """Calculate source function at given time and location."""
    global cdtds, ppw
    if ppw <= 0:
        print("ezInc: ezIncInit() must be called before ezInc.\n"
              "       Points per wavelength must be positive.\n")
        sys.exit(-1)

    arg = np.pi * ((cdtds * time - location) / ppw - 1.0)
    arg = arg * arg

    return (1.0 - 2.0 * arg) * np.exp(-arg)

In [ ]:
def updateH2d(g):
    """Update magnetic field."""
    if g.Type is oneDGrid:
        for mm in range(g.SizeX-1):
            g.Hy[mm] = g.Chyh[mm] * g.Hy[mm] \
                + g.Chye[mm] * (g.Ez[mm + 1] - g.Ez[mm])
    else:
        for mm in range(g.SizeX):
            for nn in range(g.SizeY-1):
                g.Hx[mm, nn] = g.Chxh[mm, nn] * g.Hx[mm, nn] \
                    - g.Chxe[mm, nn] * (g.Ez[mm, nn + 1] - g.Ez[mm, nn])

        for mm in range(g.SizeX-1):
            for nn in range(g.SizeY):
                g.Hy[mm, nn] = g.Chyh[mm, nn] * g.Hy[mm, nn] \
                    + g.Chye[mm, nn] * (g.Ez[mm + 1, nn] - g.Ez[mm, nn])


def updateE2d(g):
    """Update electric field."""
    if g.Type is oneDGrid:
        for mm in range(1, g.SizeX-1):
            g.Ez[mm] = g.Ceze[mm] * g.Ez[mm] \
                + g.Cezh[mm] * (g.Hy[mm] - g.Hy[mm - 1])
    else:
        for mm in range(1, g.SizeX-1):
            for nn in range(1, g.SizeY-1):
                g.Ez[mm, nn] = g.Ceze[mm, nn] * g.Ez[mm, nn] + \
                    g.Cezh[mm, nn] * ((g.Hy[mm, nn] - g.Hy[mm - 1, nn]) -
                                      (g.Hx[mm, nn] - g.Hx[mm, nn - 1]))

### Animation Setup

Functions to create animations of the 2D FDTD solvers.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.animation as animation
from matplotlib import pyplot as plt, cm
from IPython import display
from IPython.display import HTML
from matplotlib import colors
plt.style.use('classic')
import numpy as np

# Simulation parameters
SIZEX = 101
SIZEY = 81
MAXTIME = 300

# Set maximum animation file size
matplotlib.rcParams['animation.embed_limit'] = 40 * 2**20

# Defaults in raw2image.m
z_norm = 1 
decades = 3

# Create figure
fig, ax = plt.subplots(dpi=144)
im = ax.imshow(np.zeros((SIZEY, SIZEX)), cmap=cm.jet) # Transpose matrix 
plt.colorbar(im, ax=ax)
im.set_clim(-decades, 0)

def log_norm(data):
    """Return log normalized matrix."""
    return np.log10(np.abs((data)/z_norm)+np.nextafter(0, 1))
    
    
def anim_init():
    """Initialize plot for the animation."""
    im.set_data(np.zeros((SIZEY, SIZEX)))
    return (im,)


def animate(*args):
    """Draw the E-field and H-field magnitudes at current time step."""
    Ez, Hx, Hy = next(sim_step)
    data = Ez.transpose()
    data = log_norm(data)
    im.set_data(data)
    return (im,)


def html_video(frames):
    """Jupyter notebook must have animation converted to HTML to display."""
    mpl_animation = animation.FuncAnimation(
        fig, animate, frames=frames, init_func=anim_init, interval=25,
        blit=True
    )
    return mpl_animation


### TFSF Source

Replicate the animation shown in Figure 8.6 by creating a TFSF source offset by 5 nodes from the edges of the grid.

In [ ]:
def tmzdemo2():
    """TMz simulation with TFSF source at left side of grid."""
    
    g = Grid()

    gridInit(g)        # initialize the grid
    abcInit(g)         # initialize ABC
    tfsfInit(g)        # initialize TFSF boundary
    
    # do time stepping
    for g.Time in range(g.MaxTime):
        updateH2d(g)      # update magnetic field
        tfsfUpdate(g)     # apply TFSF boundary
        updateE2d(g)      # update electric field
        abc(g)            # apply ABC
        yield g.Ez, g.Hx, g.Hy

In [ ]:
sim_step = tmzdemo2()

anim = html_video(MAXTIME)
HTML(anim.to_jshtml())

### Vertical PEC Plate

Replicate the animation shown in Figure 8.7 by zeroing out Ez in a vertical line 41 nodes long, and offset 20 nodes from the left, bottom, and top of the grid.

In [ ]:
fname2 = "animations/tmzdemo2_plate.html"

pec_left_offset = 20
pec_bottom_offset = 20
pec_top_offset = 20

def add_PEC_plate(g):
    for nn in range(pec_bottom_offset, g.SizeY - pec_top_offset):
        g.Ceze[pec_left_offset, nn] = 0
        g.Cezh[pec_left_offset, nn] = 0

def tmzdemo2_plate():
    """TMz simulation with TFSF source at left side of grid and a vertical PEC plate."""
    
    g = Grid()

    gridInit(g)        # initialize the grid
    add_PEC_plate(g)   # add vertical PEC plate to grid
    abcInit(g)         # initialize ABC
    tfsfInit(g)        # initialize TFSF boundary
    
    # do time stepping
    for g.Time in range(g.MaxTime):
        updateH2d(g)      # update magnetic field
        tfsfUpdate(g)     # apply TFSF boundary
        updateE2d(g)      # update electric field
        abc(g)            # apply ABC
        yield g.Ez, g.Hx, g.Hy

In [ ]:
z_norm = 2 # Per page 208 of section 8.6 
sim_step = tmzdemo2_plate()

anim = html_video(MAXTIME)
HTML(anim.to_jshtml())

### PEC Circle

Replicate the animation shown in Figure 8.14 by zeroing out Ez in a circular pattern.  The text uses the TE<sup>Z</sup> solver for this example, but the code below still uses the TM<sup>Z</sup> solver.

In [ ]:
fname3 = "animations/tmzdemo2_circle.html"

def add_PEC_circle(g):
    rad = 12
    xCenter = g.SizeX // 2
    yCenter = g.SizeY // 2
    for mm in range(1, g.SizeX-1):
        xLocation = mm - xCenter
        for nn in range(1, g.SizeY-1):
            yLocation = nn - yCenter
            if xLocation**2 + yLocation**2 < rad**2:
                g.Ceze[mm, nn] = 0
                g.Cezh[mm, nn] = 0

def tmzdemo2_circle():
    """TMz simulation with TFSF source at left side of grid and a PEC circle."""
    
    g = Grid()

    gridInit(g)        # initialize the grid
    add_PEC_circle(g)  # add PEC circle to grid
    abcInit(g)         # initialize ABC
    tfsfInit(g)        # initialize TFSF boundary
    
    # do time stepping
    for g.Time in range(g.MaxTime):
        updateH2d(g)      # update magnetic field
        tfsfUpdate(g)     # apply TFSF boundary
        updateE2d(g)      # update electric field
        abc(g)            # apply ABC
        yield g.Ez, g.Hx, g.Hy

In [ ]:
z_norm = 2
sim_step = tmzdemo2_circle()

anim = html_video(200)
HTML(anim.to_jshtml())